<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%AA%A8%EB%8D%B8%EB%A7%81/%EC%84%B1%EC%9D%B8%EC%86%8C%EB%93%9D%EC%98%88%EC%B8%A1%EB%AA%A8%EB%8D%B8_groupby_fillna_lambda_mode_get_dummies_LabelEncoder_RandomForestClassifeir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-/view?usp=sharing
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-
To: /content/kaggle.json
100% 73.0/73.0 [00:00<00:00, 22.0kB/s]


In [2]:
!kaggle datasets download -d uciml/adult-census-income

  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 106MB/s]


In [3]:
!unzip /content/adult-census-income.zip

Archive:  /content/adult-census-income.zip
  inflating: adult.csv               


In [407]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("/content/adult.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='income', null_name='?')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26048, 15), (6513, 15), (26048, 2), (6513, 2))

-------

# EDA

In [408]:
X_train

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
21851,21851,36,Private,241998,Bachelors,13,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,50,United-States
7632,7632,53,Private,103950,Masters,14,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States
27878,27878,19,Private,203061,Some-college,10,Never-married,Tech-support,Not-in-family,White,Female,0,0,25,United-States
14121,14121,20,Private,102607,HS-grad,9,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,30,United-States
32345,32345,54,State-gov,138852,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2669,2669,45,Private,187370,Masters,14,Divorced,Exec-managerial,Unmarried,White,Male,7430,0,70,United-States
17536,17536,36,Private,174308,11th,7,Divorced,Transport-moving,Not-in-family,White,Male,0,0,40,United-States
6201,6201,47,Private,275361,Assoc-acdm,12,Widowed,Other-service,Own-child,White,Female,0,0,35,United-States
27989,27989,50,Self-emp-not-inc,196504,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,23,United-States


라벨로 되어 있는 데이터가 많은 것을 확인

In [409]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26048 entries, 21851 to 25716
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              26048 non-null  int64 
 1   age             26048 non-null  int64 
 2   workclass       24592 non-null  object
 3   fnlwgt          26048 non-null  int64 
 4   education       26048 non-null  object
 5   education.num   26048 non-null  int64 
 6   marital.status  26048 non-null  object
 7   occupation      24585 non-null  object
 8   relationship    26048 non-null  object
 9   race            26048 non-null  object
 10  sex             26048 non-null  object
 11  capital.gain    26048 non-null  int64 
 12  capital.loss    26048 non-null  int64 
 13  hours.per.week  26048 non-null  int64 
 14  native.country  25587 non-null  object
dtypes: int64(7), object(8)
memory usage: 3.2+ MB


In [410]:
X_train.isna().sum()

id                   0
age                  0
workclass         1456
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1463
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     461
dtype: int64

데이터 크기에 비해 결측치가 너무 많은 것을 확인.   
결측치 처리에 고민을 많이 해야할 듯 함.

In [411]:
X_train.nunique()

id                26048
age                  72
workclass             8
fnlwgt            18383
education            16
education.num        16
marital.status        7
occupation           14
relationship          6
race                  5
sex                   2
capital.gain        115
capital.loss         88
hours.per.week       94
native.country       41
dtype: int64

In [412]:
X_train['workclass'].value_counts()

Private             18160
Self-emp-not-inc     2049
Local-gov            1648
State-gov            1037
Self-emp-inc          909
Federal-gov           770
Without-pay            12
Never-worked            7
Name: workclass, dtype: int64

In [413]:
X_train['occupation'].value_counts()

Exec-managerial      3323
Prof-specialty       3306
Craft-repair         3296
Adm-clerical         3037
Sales                2898
Other-service        2624
Machine-op-inspct    1584
Transport-moving     1257
Handlers-cleaners    1080
Farming-fishing       786
Tech-support          746
Protective-serv       521
Priv-house-serv       119
Armed-Forces            8
Name: occupation, dtype: int64

In [414]:
X_train['race'].value_counts()

White                 22270
Black                  2484
Asian-Pac-Islander      832
Amer-Indian-Eskimo      247
Other                   215
Name: race, dtype: int64

In [415]:
X_train['native.country'].value_counts()

United-States                 23381
Mexico                          516
Philippines                     158
Germany                         108
Canada                           88
Puerto-Rico                      87
El-Salvador                      76
India                            73
Cuba                             73
England                          69
Italy                            63
South                            62
Jamaica                          59
Vietnam                          57
China                            57
Guatemala                        54
Dominican-Republic               51
Japan                            49
Poland                           47
Columbia                         44
Taiwan                           37
Haiti                            37
Iran                             34
Portugal                         32
Peru                             29
Nicaragua                        27
Ecuador                          25
Greece                      

In [416]:
X_train[X_train['native.country'].isna()]['race'].value_counts()

White                 302
Black                  73
Asian-Pac-Islander     71
Other                  15
Name: race, dtype: int64

--------

# 결측치 처리

데이터 간의 특정한 경향성이 파악되지 않는 만큼 임의로 결측치를 처리하는 것은 좋지 않다 생각함.   
채워야하는 occupation과 native.country의 경우 교육수준, 인종, 성별에 따라 경향성이 나타날 것이라 생각하여 복잡한 방식으로 groupby를 활용하여 최빈값을 구하기로 결정.   
groupby의 조건이 많이 붙는 만큼 그룹화 과정에서 해당하는 최빈값이 없어 오류가 발생.   
오류를 방지하기 위해 최빈값이 도출될 때만 연산이 진행되도록 테스트.

In [417]:
X_train_groupby_native_country = X_train.groupby(['education.num', 'race', 'sex'])['native.country'].apply(lambda x : x.mode().iloc[0] if x.mode().empty == False else 0)

In [418]:
X_train_groupby_native_country

education.num  race                sex   
1              Asian-Pac-Islander  Female         Cambodia
                                   Male              China
               Black               Female    United-States
                                   Male      United-States
               Other               Male             Mexico
                                                 ...      
16             Black               Female    United-States
                                   Male      United-States
               Other               Male      United-States
               White               Female    United-States
                                   Male      United-States
Name: native.country, Length: 151, dtype: object

In [419]:
X_train_groupby_occupation = X_train.groupby(['education.num', 'race', 'sex'])['occupation'].apply(lambda x : x.mode().iloc[0] if x.mode().empty == False else 0)

In [420]:
X_train_groupby_occupation

education.num  race                sex   
1              Asian-Pac-Islander  Female    Machine-op-inspct
                                   Male           Craft-repair
               Black               Female         Adm-clerical
                                   Male      Machine-op-inspct
               Other               Male          Other-service
                                                   ...        
16             Black               Female       Prof-specialty
                                   Male         Prof-specialty
               Other               Male           Adm-clerical
               White               Female       Prof-specialty
                                   Male         Prof-specialty
Name: occupation, Length: 151, dtype: object

In [421]:
X_train_groupby_occupation.value_counts()

Other-service        31
Prof-specialty       30
Craft-repair         27
Adm-clerical         23
Machine-op-inspct    16
Exec-managerial       7
Transport-moving      4
Handlers-cleaners     4
Sales                 4
Farming-fishing       3
Priv-house-serv       1
0                     1
Name: occupation, dtype: int64

해당 방식으로 값이 제대로 구해지는 것을 확인하고 fillna()에 도입

In [422]:
X_train_id = X_train['id']
X_train = X_train.drop(columns='id', axis=1)
X_train = X_train.drop(columns=['workclass', 'education'], axis=1)
X_train['occupation'] = X_train.groupby(['education.num', 'race', 'sex'])['occupation'].apply(lambda x : x.fillna(x.mode().iloc[0]) if x.mode().empty == False else x.fillna('Empty'))
X_train['native.country'] = X_train.groupby(['education.num', 'race', 'sex'])['native.country'].apply(lambda x : x.fillna(x.mode().iloc[0]) if x.mode().empty == False else x.fillna('Empty'))

In [423]:
X_train.isna().sum()

age               0
fnlwgt            0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
dtype: int64

결측치가 사라진 것을 확인

하지만 fillna를 진행하며 최빈값이 검색이 안 될 경우 Empty를 넣도록 했기에 Empty에 해당되는 데이터는 삭제하도록함.

In [424]:
drop_index = X_train[(X_train['occupation'] == 'Empty') | (X_train['native.country'] == 'Empty')].index

In [425]:
drop_index

Int64Index([26938], dtype='int64')

In [400]:
X_train = X_train.drop(index=drop_index, axis=1)
y_train = y_train.drop(index=drop_index, axis=1)
print(X_train.shape)
print(y_train.shape)

(26047, 12)
(26047, 2)


해당 과정을 X_test와 y_test에 대해서도 진행

In [426]:
X_test_id = X_test['id']
X_test = X_test.drop(columns='id', axis=1)
X_test = X_test.drop(columns=['workclass', 'education'], axis=1)
X_test['occupation'] = X_test.groupby(['education.num', 'race', 'sex'])['occupation'].apply(lambda x : x.fillna(x.mode().iloc[0]) if x.mode().empty == False else x.fillna('Empty'))
X_test['native.country'] = X_test.groupby(['education.num', 'race', 'sex'])['native.country'].apply(lambda x : x.fillna(x.mode().iloc[0]) if x.mode().empty == False else x.fillna('Empty'))
drop_index_test = X_test[(X_test['occupation'] == 'Empty') | (X_test['native.country'] == 'Empty')].index
X_test = X_test.drop(index=drop_index_test, axis=1)
y_test = y_test.drop(index=drop_index_test, axis=1)

-------

# 원핫인코딩 및 라벨인코딩
두 개의 카테고리로 나뉘는 sex에 대해서 원핫인코딩 진행.   
카테고리가 많은 다른 행에 대해서는 라벨인코딩 진행.   

In [428]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
X_train['sex'] = pd.get_dummies(X_train['sex'], drop_first=True)
X_test['sex'] = pd.get_dummies(X_test['sex'], drop_first=True)
cols = ['marital.status', 'occupation', 'relationship', 'race', 'native.country']
for col in cols:
    X_train[col] = LE.fit_transform(X_train[col])
    X_test[col] = LE.transform(X_test[col])

------

# y 정제
y의 타겟 값이 str형태이므로 50K 초과를 1로, 50K 이하를 0으로 치환

In [430]:
y_train

,id,income
21851,21851,>50K
7632,7632,<=50K
27878,27878,<=50K
14121,14121,<=50K
32345,32345,<=50K
...,...,...
2669,2669,>50K
17536,17536,<=50K
6201,6201,<=50K
27989,27989,<=50K


In [432]:
y_train['income'] = y_train['income'].replace({'>50K': 1, '<=50K' : 0})

In [434]:
y_test['income'] = y_test['income'].replace({'>50K': 1, '<=50K' : 0})

------

# 모델링 : Random Forest Classifier
많은 수의 라벨 인코딩이 진행되었고 결과값은 둘 중 하나이므로 Random Forest Classifer를 활용하기로 결정.

In [431]:
from sklearn.ensemble import RandomForestClassifier

In [436]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=4, min_samples_leaf=20, random_state=2022)
model.fit(X_train, y_train['income'])
print(model.score(X_train, y_train['income']))
print(model.score(X_test, y_test['income']))

0.8510058353808354
0.8518290808484476
